In [1]:
import sys
sys.path.insert(0, '/home/shayan/phoenix/graphite/')

In [2]:
import argparse
import wandb
import torch
import torch.cuda
import torch.distributed
from graphite.utilities.config.manager.utilities import read_config
from graphite.utilities.device import get_device
from graphite.utilities.miscellaneous import count_parameters
from graphite.utilities.randomization.seed import fix_random_seeds
from graphite.utilities.argument_parsing.train import base_args, distributed_args
from graphite.utilities.distributed.utilities import setup_distributed_training_if_requested
import graphite.data.handler as data_handler_lib
import graphite.cortex.optimization.optimizer as optimizer_lib
import graphite.cortex.optimization.special as special_optimization_lib
import graphite.cortex.optimization.scheduler as scheduler_lib
import graphite.cortex.model as model_lib
import graphite.cortex.trainer as trainer_lib
from graphite.utilities.wandb.utilities import initialize_wandb
from graphite.utilities.miscellaneous import count_parameters
from graphite.utilities.logging import get_logger, log_message

/home/shayan/anaconda3/envs/marrovision/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-13 18:51:50,087 - rdkit - INFO - Enabling RDKit 2022.03.5 jupyter extensions


In [13]:
args = argparse.Namespace()
args.config = '/home/shayan/phoenix/graphite/lab/configs/experiment/pcqm4mv2/2d/grpe_advanced/exp5.py'
args.config_overrides=None
args.distributed  =  False

In [31]:
config = read_config(args)

In [15]:
data_handler = getattr(data_handler_lib, config['data']['type'])(
        distributed=args.distributed,
        distributed_sampling='all',
        **config['data']['args']
    )

In [25]:
dataloaders = data_handler.get_dataloaders()

In [18]:
batch = next(iter(dataloaders['train']))

In [33]:
model = getattr(model_lib, config['model']['type'])( **config['model']['args'])
count_parameters(model)

113956601

In [34]:
%%time
loss, outputs = model(batch)

CPU times: user 1min 47s, sys: 15.8 s, total: 2min 2s
Wall time: 6.33 s


In [35]:
config['model']['args']['model_config']['args']['toeplitz'] = True
model = getattr(model_lib, config['model']['type'])( **config['model']['args'])
count_parameters(model)

113960505

In [36]:
%%time
loss, outputs = model(batch)

CPU times: user 1min 52s, sys: 3.86 s, total: 1min 56s
Wall time: 6.11 s


In [39]:
for n, p in model.named_parameters():
    if 'toeplitz' in n:
        print(n)
        print(f'\t {p.shape}')

model.toeplitz_row
	 torch.Size([32, 61])
model.toeplitz_col
	 torch.Size([32, 61])
